Step 1: Define Social Media Spikes
Identify spike days/weeks/months where social media post volume exceeds the rolling historical mean by more than two standard deviations, indicating statistically significant bursts of attention.

In [ ]:
import pandas as pd

# Load and melt dataset as before
df = pd.read_csv('bluesky_allkeywords.csv')

df_long = pd.melt(
    df,
    id_vars=['company'],
    var_name='date',
    value_name='all_keywords_mentions'
)
df_long['date'] = pd.to_datetime(df_long['date'])

# Mapping company to ticker explicitly
sp500_tickers = ["WMT", "AMZN", "AAPL", "CVS", "TSLA", "GOOGL", "META", "JPM", "COST", "KR",
"WBA", "TGT", "UPS", "CNC", "CI", "MSFT", "VZ", "IBM", "UNH", "XOM",
"MCK", "CVX", "CAH", "HD", "WBA", "MPC", "COR", "F", "C", "DELL",
"GM", "NVDA", "PFE", "PG", "CMCSA", "JNJ", "WFC", "BAC", "ABBV", "SBUX",
"CSCO", "T", "PEP", "INTC", "DIS", "BA", "GS", "MS", "HON", "CRM"
]
company_names = df_long['company'].unique()
company_to_ticker = dict(zip(company_names, sp500_tickers))
df_long['ticker'] = df_long['company'].map(company_to_ticker)

# Drop the original company column
df_long = df_long.drop(columns=['company'])

# Sort for rolling calculation
df_long = df_long.sort_values(['ticker', 'date']).reset_index(drop=True)


         date  all_keywords_mentions ticker
0  2024-08-01                   1363   AAPL
1  2024-08-02                   1409   AAPL
2  2024-08-03                   1277   AAPL
3  2024-08-04                   1221   AAPL
4  2024-08-05                   1347   AAPL
5  2024-08-06                   1322   AAPL
6  2024-08-07                   1373   AAPL
7  2024-08-08                   1289   AAPL
8  2024-08-09                   1322   AAPL
9  2024-08-10                   1147   AAPL
10 2024-08-11                   1044   AAPL
11 2024-08-12                   2176   AAPL
12 2024-08-13                   1647   AAPL
13 2024-08-14                   1579   AAPL
14 2024-08-15                   1452   AAPL
15 2024-08-16                   1467   AAPL
16 2024-08-17                   1216   AAPL
17 2024-08-18                   1285   AAPL
18 2024-08-19                   1470   AAPL
19 2024-08-20                   1421   AAPL
20 2024-08-21                   1361   AAPL
21 2024-08-22                   

In [76]:

# Spike flagging function
def flag_spikes(df, window_size, count_col, flag_col):
    rolling_mean = df.groupby('ticker')[count_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    rolling_std = df.groupby('ticker')[count_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).std().fillna(0))
    df[flag_col] = df[count_col] > (rolling_mean + 1.5 * rolling_std)
    return df


# Daily spikes
df_long = flag_spikes(df_long, 7, 'all_keywords_mentions', 'daily_spike')
daily_spikes = df_long[df_long['daily_spike']][['date', 'ticker', 'all_keywords_mentions']]

# Weekly spikes
df_weekly = df_long.set_index('date').groupby('ticker')['all_keywords_mentions'].resample('W').sum().reset_index()
df_weekly = df_weekly.sort_values(['ticker', 'date']).reset_index(drop=True)
df_weekly = flag_spikes(df_weekly, 21, 'all_keywords_mentions', 'weekly_spike')
weekly_spikes = df_weekly[df_weekly['weekly_spike']][['date', 'ticker', 'all_keywords_mentions']]

# Monthly spikes
df_monthly = df_long.set_index('date').groupby('ticker')['all_keywords_mentions'].resample('M').sum().reset_index()
df_monthly = df_monthly.sort_values(['ticker', 'date']).reset_index(drop=True)
df_monthly = flag_spikes(df_monthly, 30, 'all_keywords_mentions', 'monthly_spike')
monthly_spikes = df_monthly[df_monthly['monthly_spike']][['date', 'ticker', 'all_keywords_mentions']]

print(monthly_spikes)

          date ticker  all_keywords_mentions
18  2025-02-28   ABBV                    220
20  2025-04-30   ABBV                    289
21  2025-05-31   ABBV                    318
23  2025-07-31   ABBV                    450
45  2025-05-31     BA                  50827
58  2025-06-30    BAC                   6468
102 2025-02-28  CMCSA                   4425
162 2025-02-28   CSCO                   2173
258 2025-02-28     GS                   8106
307 2025-03-31   INTC                  27792
365 2025-01-31   META                 269928
379 2025-03-31    MPC                   1254
401 2025-01-31   MSFT                  50940
413 2025-01-31   NVDA                  26116
436 2024-12-31    PFE                   2488
460 2024-12-31   SBUX                  17332
498 2025-02-28   TSLA                 255432
499 2025-03-31   TSLA                 290416
508 2024-12-31    UNH                  52068
534 2025-02-28     VZ                   6054
545 2025-01-31    WBA                   8086
556 2024-1

C:\Users\Thisu\AppData\Local\Temp\ipykernel_17244\793306762.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df_long.set_index('date').groupby('ticker')['all_keywords_mentions'].resample('M').sum().reset_index()


Step 2: Estimate Post-Spike Returns and Volatility

To estimate log returns, use the ??????
To estimate weekly volatility, use the average weekly volatility for the previous 5 days.
To estimate monthly volatility, use the average monthly volatility for the previous 21 days. 

In [ ]:
###FIX CODE, LOAD IN DATA, THEN DO VOLATILITY ESTIMATION
S


##Code to get corresponding trading days before/after spikes
import pandas as pd
import pandas_market_calendars as mcal
import numpy as np

#Now, to find trading day info for spikes
#Make an estimation for "expected" weekly volatility and monthly volatility using rolling averages of historical data
stock_data = pd.read_csv('bluesky_stock_merged.csv') 
stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.normalize()

#Function to get row values from stock_data DataFrame
def get_value(df, index, column_name):
    row = df.loc[index]
    return row[column_name]


columns = ['Date', 'Ticker', 'Spike', 'AbnormalVolatility']
weekly_sigma_spike = pd.DataFrame(columns=columns)

#Weekly volatility estimation
for inx in weekly_spikes.index:
    row = weekly_spikes.loc[inx]
    date = row['date']
    ticker = row['company']
    historical_weekly_volatility = [get_value(stock_data, inx - i, 'weekly_variance') for i in [1,2,3,4,5]]
    expected_weekly_volatility = pd.Series(historical_weekly_volatility).mean()
    actual_volatility = get_value(stock_data, inx + 5, 'weekly_variance')
    abnormal_volatility = np.abs(actual_volatility - expected_weekly_volatility) if actual_volatility and expected_weekly_volatility else None
    indicator = 1
    new_row = pd.DataFrame({
        'Date': [date],
        'Ticker': [ticker],
       'WeeklyVariance': [expected_weekly_volatility],
        'Spike':[indicator],
        'AbnormalVolatility': [abnormal_volatility]
    })
    weekly_sigma_spike = pd.concat([weekly_sigma_spike, new_row], ignore_index=True)


#repeat for monthly volatility estimation
columns = ['Date', 'Ticker', 'Spike', 'AbnormalVolatility']
monthly_sigma_spike = pd.DataFrame(columns=columns)

for inx in monthly_spikes.index:
    row = monthly_spikes.loc[inx]
    date = row['date']
    ticker = row['company']
    historical_monthly_volatility = [get_value(stock_data, inx - i, 'monthly_variance') for i in [1,2,3,4,5,6,7]]
    expected_monthly_volatility = pd.Series(historical_monthly_volatility).mean()
    actual_volatility = get_value(stock_data, inx + 21, 'monthly_variance')
    abnormal_volatility = np.abs(actual_volatility - expected_monthly_volatility) if actual_volatility and expected_monthly_volatility else None
    indicator = 1
    new_row = pd.DataFrame({
        'Date': [date],
        'Ticker': [ticker],
       'MonthlyVariance': [expected_monthly_volatility],
        'Spike':[indicator],
        'AbnormalVolatility': [abnormal_volatility]
    })
    monthly_sigma_spike = pd.concat([monthly_sigma_spike, new_row], ignore_index=True)

#repeat for daily volatility estimation
columns = ['Date', 'Ticker', 'Spike', 'AbnormalVolatility']
daily_sigma_spike = pd.DataFrame(columns=columns)

for inx in daily_spikes.index:
    row = daily_spikes.loc[inx]
    date = row['date']
    ticker = row['company']
    historical_daily_volatility = [get_value(stock_data, inx - i, 'log_return') for i in [1,2,3,4,5,6,7,8,9,10]]
    expected_daily_volatility = pd.Series(historical_daily_volatility).mean()
    actual_volatility = get_value(stock_data, inx + 1, 'log_return')
    abnormal_volatility = np.abs(actual_volatility - expected_daily_volatility) if actual_volatility and expected_daily_volatility else None
    indicator = 1
    new_row = pd.DataFrame({
        'Date': [date],
        'Ticker': [ticker],
       'DailyVariance': [expected_daily_volatility],
        'Spike':[indicator],
        'AbnormalVolatility': [abnormal_volatility]
    })
    daily_sigma_spike = pd.concat([daily_sigma_spike, new_row], ignore_index=True)

print(daily_sigma_spike)

C:\Users\Thisu\AppData\Local\Temp\ipykernel_17244\2395438683.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weekly_sigma_spike = pd.concat([weekly_sigma_spike, new_row], ignore_index=True)
C:\Users\Thisu\AppData\Local\Temp\ipykernel_17244\2395438683.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  monthly_sigma_spike = pd.concat([monthly_sigma_spike, new_row], ignore_index=True)
C:\Users\Thisu\AppData\Local\Temp\ipykernel_17244\2395438683.py:82: FutureWarning: The behavior of DataFrame

KeyError: 9468

Step 3: Compare realized volatility/returns with estimated returns/volatility


Step 4: Regress abnormal returns and abnormal volatility on social media spike indicators controlling for market index